In [ ]:
import pandas as pd
import jsonlines
from geopy.distance import distance
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [ ]:
df_metro = pd.read_json("../data/input/metro.jsonlines", lines=True)
df_metro

In [ ]:
data = []
with jsonlines.open('../data/input/BR_Rio_de_Janeiro_NULL_Rio_de_Janeiro_Zona_Oeste_Barra_da_Tijuca.jsonl') as reader:
    for obj in reader:
        data.append(obj)

In [ ]:
df = pd.json_normalize(data, sep='_')
df.sample(2)

In [ ]:
df = pd.json_normalize(data, sep="_")
df.loc[:, "media"] = df.medias.apply(
    lambda x: [i["url"].format(action="fit-in", width=870, height=653) for i in x]
)
df.loc[:, "created_at"] = pd.to_datetime(df.listing_createdAt)
df.loc[:, "updated_at"] = pd.to_datetime(df.listing_updatedAt)
df.loc[:, "usable_area"] = df.listing_usableAreas.str[0].astype(int)
df.loc[:, "floors"] = df.listing_floors.str[0].fillna(-1)
df.loc[:, "units_on_the_floor"] = df.listing_unitsOnTheFloor.fillna(-1)
df.loc[:, "unit_floor"] = df.listing_unitFloor.fillna(-1)
df.loc[:, "type_unit"] = df.listing_unitTypes.str[0]
df.loc[:, "bedrooms"] = df.listing_bedrooms.str[0]
df.loc[:, "bathrooms"] = df.listing_bathrooms.str[0]
df.loc[:, "suites"] = df.listing_suites.str[0].fillna(0)
df.loc[:, "parking_spaces"] = df.listing_parkingSpaces.str[0].fillna(0)

In [ ]:
df = pd.concat(
    [
        df,
        df.listing_pricingInfos.apply(
            lambda x: [
                {
                    "price": p.get("price", 0),
                    "condo_fee": p.get("monthlyCondoFee", 0),
                }
                for p in x
                if p["businessType"] == "RENTAL"
            ][0]
        )
        .apply(pd.Series)
        .astype(int),
    ],
    axis=1,
).assign(total_fee=lambda x: x["price"] + x["condo_fee"])

fl_zap = df["listing_portal"] == "GRUPOZAP"
df.loc[fl_zap, "origin"] = "zapimoveis"
df.loc[~fl_zap, "origin"] = "vivareal"

df.loc[:, "url"] = df[["origin", "link_href"]].apply(
    lambda x: "https://www.{}.com.br{}".format(*x), axis=1
)

In [ ]:
df[df.bedrooms.isna()].url

In [ ]:
df[df.url.str.contains("2495611515")]

In [ ]:
df.listing_bathrooms

In [ ]:
df.listing_usableAreas.str[0]

In [ ]:
t = df[df.listing_pricingInfos.apply(lambda x: len(x)) == 2].listing_pricingInfos[41]
t

In [ ]:
t[1]

In [ ]:
[a for a in t if a["businessType"] == 'RENTAL']

In [ ]:
df

In [ ]:
pd.concat(
        [
            df,
            df.listing_pricingInfos.apply(
                lambda x: [
                    {
                        "price": p.get("price", 0),
                        "condo_fee": p.get("monthlyCondoFee", 0),
                    }
                    for p in x
                    if p["businessType"] == "RENTAL"
                ][0]
            ).apply(pd.Series).astype(int)
        ],
        axis=1,
    ).dtypes

In [ ]:
df.listing_pricingInfos.apply(lambda x: [{"price": p.get("price", 0), "condo_fee": p.get("monthlyCondoFee", 0)} for p in x if p["businessType"] == 'RENTAL'][0]).apply(pd.Series).astype(int)

In [ ]:
df.listing_unitsOnTheFloor

In [ ]:
df.listing_pricingInfos[0]

In [ ]:
df.medias.apply(lambda x: [i["url"].format(action="fit-in", width=870, height=653) for i in x])

In [ ]:
def get_min_dist(lat, lng):
    if not pd.isna(lat):
        X = df_metro.copy()
        X["distance"] =  X[["lat", "lng"]].apply(lambda x: distance((x[0], x[1]), (lat, lng)).m, axis=1)
        return X.sort_values("distance").reset_index().loc[0,['linha', 'estacao', 'distance']].tolist()
        #return X.sort_values("distance").reset_index()
    else:
        return "", "", None

In [ ]:
df[['lat', 'lng','condominio']] = df[['lat', 'lng','condominio']].apply(pd.to_numeric)

In [ ]:
get_min_dist(-22.925214,-43.230616)

In [ ]:
df[['linha', 'estacao', 'distance']] = df[["lat", "lng"]].apply(lambda x: get_min_dist(x[0], x[1]), axis=1).apply(pd.Series)

In [ ]:
df[df.url == "https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-65m2-aluguel-RS1550-id-92895851/?__vt=lgpd:b"]

In [ ]:
df[~df.lat.isna()]

In [ ]:
df.groupby("page")["id"].count()

In [ ]:
df.groupby("tipo")["id"].count()

In [ ]:
df = df[df.tipo == "Aluguel"]
df.head(3)

In [ ]:
df["preco_total"] = df["preco"] + df["condominio"]

In [ ]:
df.head(3)